### 数据集的介绍 
官网下载地址：https://www.kaggle.com/c/dogs-vs-cats/data   
官方文件介绍：  
训练集： 25000张图像，图像名称格式cat.数字.jpg或者dog.数字.jpg相当于标签，图像尺寸大小不一，彩色图像。12500张cat图像，12500张dog图像，图像按顺序排列。  
训练集： 12500张图像，图像名称格式cat.数字.jpg或者dog.数字.jpg，相当于无标签，图像尺寸大小不一，彩色图像。图像打乱。  
实验说明：  
由于硬件条件和时间有限，实验只取部分数据！！   若你条件允许，程序无需修改可直接使用！！   
1 取官方训练集的cat前640张和dog前640张作为新的训练集，存放在文件夹名为sdh_train下。 0-639   
2 取官方训练集去除新训练集后的cat前360张和dog前360张作为测试集,存放在文件夹名为sdh_test下。 640-999   
3 新的训练集和测试集都含有标签，共2000张图像。   
4 程序在Google colab上运行，非windows本地。  
5 本节仅仅展示训练过程结果，下节将进行测试结果展示和分析。     

### 使用pytorch读取本地图像的关键问题 
1 如何获取每个图像的内容和对应标签？   
2 如何将获所有取到的图像内容与标签组成的数据集分类batch？  


### pytorch读取本地图像问题的解决方法 
1 熟悉流程:   
pytoch中所有数据集datasets都要作为torch.utils.data.Dataset的子类，即任何数据集类都要继承于Dataset类，并且要覆写Dataset类的\_\_getitem\_\_方法和\_\_len\_\_方法，同时也要写上自己的\_\_init\_\_方法。  
\_\_getitem\_\_方法目的是获取图像内容和对应的标签值。  
\_\_len\_\_方法的目的是计算数据集的大小，后续根据batch_size的大小，计算batch的个数。   
\_\_init\_\_方法的目的首先是为每张图像打标签，然后将所有图像的图像路径+图像名和对应的所有标签分类存放在2个属性中，属性为list类型。    

2 分步操作：  
(1) 定义自己的数据集类必须要继承torch.utils.data.Dataset类    
(2) 实现\_\_init\_\_方法，定义2个属性，list类型，用于存放图像名和对应的标签。遍历文件夹读取每张图像的名称，根据名称打标签 ，分别添加到定义好的2个属性列表中。   
(3) 覆写Dataset类的\_\_getitem\_\_方法，该方法接收一个index索引参数，利用这个index正好可以获取在\_\_init\_\_方法中得到的2个属性列表中的图像路径+图像名和标签，然后根据文件名直接读取图像内容和对应的标签。这样问题1就解决了。   
(4) pytorch将给定的数据集分成batch，直接使用DataLoader类输入参数进行即可，导入DataLoader类使用 from torch.utils.data import DataLoader。 这样问题2就解决了。


### DataLoader类是如何触发\_\_getitem\_\_方法执行图像内容和标签读取的？  
详细解释流程：  
前提：理解python中的迭代器和可迭代对象，如果不理解不要往下看！！！   
一个具备了\_\_iter\_\_方法的对象，就是一个可迭代对象，由DataLoader类创建的对象就是一个可迭代的对象。  
一个实现了\_\_iter\_\_方法和\_\_next\_\_方法的对象，就是迭代器。由_BaseDataLoaderIter子类创建的对象就是迭代器。  
python中的\_\_getitem\_\_方法可以通过对象[索引]自动调用！！！  

1 DataLoader类\_\_init\_\_方法
参数：   
self,   
dataset,   
batch_size=1,   
shuffle=False,   
sampler=None,  
batch_sampler=None，  
num_workers=0，  
collate_fn=None，  
pin_memory=False，  
drop_last=False，  
timeout=0，  
worker_init_fn=None，  
multiprocessing_context  

2 DataLoader类\_\_iter\_\_方法     
根据线程数返回不同的迭代器对象的引用：  
num_workers=0，return  _SingleProcessDataLoaderIter(self)   
否则，return  _MultiProcessingDataLoaderIter(self)   

class _SingleProcessDataLoaderIter(_BaseDataLoaderIter)或者class _MultiProcessingDataLoaderIter(_BaseDataLoaderIter)  
class _BaseDataLoaderIter(object)是一个迭代器，实现了\_\_iter\_\_和 \_\_next\_\_方法   
class _SingleProcessDataLoaderIter也是一个迭代器  
class _MultiProcessingDataLoaderIter也是一个迭代器  

3 默认单线程，使用_SingleProcessDataLoaderIter迭代器创建对象，参数正是DataLoader类创建的对象，当对DataLoader类创建的对象即可迭代对象进行迭代时会自动调用_SingleProcessDataLoaderIter迭代器的\_\_next\_\_方法，\_\_next\_\_方法方法首先获取索引，然后通过fetch函数根据索引获取数据即自动触发Dataset类中的\_\_getitem\_\_方法获取图像的内容和标签， fetch函数的核心data = [self.dataset[idx] for idx in possibly_batched_index]。

重新走一遍流程：   
(1) 创建自己的数据集类，该类继承于data.Dataset类，覆写\_\_getitem\_\_和\_\_len\_\_方法，也定义自己的\_\_init\_\_方法。使用该类创建对象。 
(2) 使用DataLoader创建对象，第一步创建的数据集对象作为参数传入，其他参数根据情况自己设置。  
(3) 数据集对象作为参数传入后，DataLoader创建的对象拥有了以数据集对象作为的属性，即一个类的对象作为另一个类的对象的属性，在DataLoader的初始化方法\_\_init\_\_可以看到self.dataset = dataset，前者是属性，后者是传入的参数即数据集对象。   
(4) 由于DataLoader类实现了\_\_iter\_\_方法，同时\_\_iter\_\_方法返回的是一个迭代器的对象(因此可以对DataLoader类的对象进行迭代)，该迭代器以DataLoader类的对象loader(该对象有一个以数据集对象的属性)为参数，单进程使用的语句return \_\SingleProcessDataLoaderIter(self)，该类继承于\_\BaseDataLoaderIter类，\_\BaseDataLoaderIter类有一个以DataLoader类的对象的属性为属性，self.dataset = loader.dataset，所以第一步创建的数据对象先是作为DataLoader类的对象的属性，然后也传递为\_\BaseDataLoaderIter类对象的属性，接着传递为\_\SingleProcessDataLoaderIter对象的属性。  
(5) 当使用for...in语句对DataLoader类的对象进行迭代时，DataLoader类iter函数会自动获取\_\_iter\_\_方法的返回值，根据第四步(4),该返回值正是一个迭代器的对象，然后该迭代器的对象会自动调用其\_\_next\_\_方法,除此之外迭代器还干了一件事创建数据集获取器，该数据集获取器根据数据类型返回一个数据集获取对象。\_\_next\_\_方法方法首先获取索引，然后利用创建的数据集获取器对象调用fetch函数，fetch函数根据索引获取数据即自动触发Dataset类中的\_\_getitem\_\_方法获取图像的内容和标签， fetch函数的核心data = [self.dataset[idx] for idx in possibly_batched_index]  
 ![jupyter](./demo34_3.png)

### 实战环节  

In [2]:
# 1 导入需要的模块
import torchvision
import torch
import torch.utils.data as data
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torch.optim import sgd, adam
from PIL import Image
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
%matplotlib inline

In [3]:
# 2 选择GPU和图像增强操作 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
transform_train = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomCrop((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

    ]
)

transform_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
)


In [4]:
# 3 训练集和测试集的上一级路径 
image_path = "/content/drive/My Drive" 
# image_path = "D:\\迅雷下载\\dogs-vs-cats-redux-kernels-edition"  # windows上的路径 

In [5]:
# 4 训练集和测试集内容以及标签获取类 
class CatDogProcessor(data.Dataset):
    def __init__(self, mode, directory):
        self.mode = mode
        self.list_img = []
        self.list_label = []
        self.data_size = 0

        if self.mode == "train":
            self.transform = transform_train
            directory = directory + "/sdh_train/"
            for file in os.listdir(directory):
                self.list_img.append(directory + file)
                self.data_size += 1
                name = file.split(sep=".")
                if name[0] == "cat":
                    self.list_label.append(0)
                elif name[0] == "dog":
                    self.list_label.append(1)

        elif self.mode == "test":
            self.transform = transform_test
            directory = directory + "/sdh_test/"
            for file in os.listdir(directory):
                self.list_img.append(directory + file)
                self.data_size += 1
                name = file.split(sep=".")
                if name[0] == "cat":
                    self.list_label.append(0)
                elif name[0] == "dog":
                    self.list_label.append(1)

    def __getitem__(self, item):
        if self.mode == "train":
            img = Image.open(self.list_img[item])
            label = self.list_label[item]
            return self.transform(img), torch.tensor([label])
        elif self.mode == "test":
            img = Image.open(self.list_img[item])
            label = self.list_label[item]
            return self.transform(img), torch.tensor([label])
        else:
            return None

    def __len__(self):
        return self.data_size

In [6]:
# 5 定义一个神经网络模型
class SelfMadeCNN(nn.Module):
    # 5个卷积层＋2个全连接层
    def __init__(self, n_hidden_1, n_hidden_2,out_dim):
        super(SelfMadeCNN, self).__init__()
        self.layer1 = nn.Sequential(
            # (224-3+2*1)/1 +1 = 224
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # 224/2 = 112
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            # (112-3+2*0)/1 +1 = 110
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (110-2)/2 +1 = 55
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer3 = nn.Sequential(
            #  55-2+2*0+1=54
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (54-2)/2 +1 = 27
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.layer4 = nn.Sequential(
            #  27-2+2*0+1=26
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (26-2)/2 +1 = 13
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer5 = nn.Sequential(
            #  13-2+2*0+1=12
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=2, stride=1, padding=0),
            nn.BatchNorm2d(64),  # 批标准化放在卷积层后面，激活层前面
            nn.ReLU(inplace=True),
            # (12-2)/2 +1 = 6
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer6 = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(6 * 6 * 64, n_hidden_1),
            nn.ReLU(inplace=True),

            nn.Dropout(p=0.5),
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(inplace=True),
            
            nn.Linear(n_hidden_2, out_dim),

        )

    def forward(self, x):
        # 卷积层
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.layer5(x)

        # 全连接层
        # 进入全连接之前要将矩阵转为向量
        x = x.view(x.size(0), -1)
        x = self.layer6(x)

        return x

In [7]:
# 6 训练结果绘制 
def plot_image(acces, train_losses):
    plt.plot(np.arange(len(acces)), acces)
    plt.title('train acc')
    plt.show()

    plt.plot(np.arange(len(train_losses)), train_losses)
    plt.title('train loss')
    plt.show()

In [8]:
# 7 训练模型 
def train():
    image_data_set = CatDogProcessor("train", image_path)
    train_data = DataLoader(image_data_set, batch_size=64, shuffle=True)
    print("train_data loaded", len(train_data))

    model = SelfMadeCNN(100, 100, 2)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = sgd.SGD(model.parameters(), lr=0.001,momentum=0.9)

    train_losses = []
    train_acces = []
    for i in range(500):
        train_loss = 0
        train_acc = 0
        model.train()
        for im, label in train_data:
            # 前向传播
            im = im.cuda()
            label = label.cuda()
            label = label.squeeze()
            out = model(im)
            loss = criterion(out, label)

            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # 记录误差
            train_loss += loss.item()
            # 计算分类的准确率
            _, pred = torch.max(out, 1)
            num_correct = (pred == label).sum().item()
            acc = num_correct / im.shape[0]
            train_acc += acc
        train_losses.append(train_loss / len(train_data))
        train_acces.append(train_acc / len(train_data))

        print('epoch: {}, Train Loss: {:.6f}, Train Acc: {:.6f}'.format(i, train_loss / len(train_data),
                                                                        train_acc / len(train_data)))

    print("train -------------------Done!")
    plot_image(train_acces, train_losses)
    torch.save(model.state_dict(), "cat_dog_model.pth")

    

![jupyter](./demo33.png)

![jupyter](./demo33_2.png)